import needed packages

In [1]:
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from xgboost import XGBClassifier
import sklearn.preprocessing as preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold,StratifiedShuffleSplit
import sklearn.linear_model as linear_model
from sklearn.metrics import mean_absolute_error, f1_score, accuracy_score, precision_score, recall_score,roc_auc_score, roc_curve, average_precision_score,precision_recall_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_columns",80)
from datetime import datetime

import dataset and clean

In [2]:
data_path = 'E:/Research/CE/ML/newdata.xlsx'
#read data from csv file
original_data = pd.read_excel(data_path)
#remove all parentheses, replace all spaces by underscore
original_data.columns = original_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
#remove all rows where crash_severity (target) is unknown
original_data = original_data[original_data.crash_severity != '99 - UNKNOWN']
#remove some NA rows
original_data = original_data[original_data.adjusted_average_daily_traffic_amount != 'No Data']
#display column titles
original_data.columns 

Index(['crash_id', 'active_school_zone_flag',
       'adjusted_average_daily_traffic_amount',
       'adjusted_percentage_of_average_daily_traffic_for_trucks',
       'at_intersection_flag', 'commercial_motor_vehicle_flag',
       'construction_zone_flag', 'county', 'crash_severity', 'crash_time',
       'crash_year', 'curve_degrees', 'curve_length', 'curve_type',
       'day_of_week', 'first_harmful_event', 'highway_lane_design',
       'inside_shoulder_width_on_divided_highway', 'intersection_related',
       'latitude', 'left_shoulder_type', 'left_shoulder_use',
       'left_shoulder_width', 'light_condition', 'longitude',
       'manner_of_collision', 'median_type', 'median_width', 'number_of_lanes',
       'object_struck', 'right_shoulder_type', 'right_shoulder_use',
       'right_shoulder_width', 'road_class', 'roadway_alignment',
       'roadway_function', 'roadway_part', 'roadway_type', 'rural_flag',
       'rural_urban_type', 'speed_limit', 'surface_type', 'surface_width',
   

In [3]:
#continue to remove some NA rows
original_data = original_data[original_data.rural_urban_type != 'No Data']
#see general information about the dataset
print(original_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8859 entries, 0 to 13947
Data columns (total 58 columns):
crash_id                                                   8859 non-null int64
active_school_zone_flag                                    8859 non-null object
adjusted_average_daily_traffic_amount                      8859 non-null object
adjusted_percentage_of_average_daily_traffic_for_trucks    8859 non-null object
at_intersection_flag                                       8859 non-null bool
commercial_motor_vehicle_flag                              8859 non-null object
construction_zone_flag                                     8859 non-null object
county                                                     8859 non-null object
crash_severity                                             8859 non-null object
crash_time                                                 8859 non-null int64
crash_year                                                 8859 non-null int64
curve_degrees    

In [4]:
#some numerical features were read as object, convert it back to numerical
original_data['adjusted_average_daily_traffic_amount']=original_data['adjusted_average_daily_traffic_amount'].apply(pd.to_numeric, errors='coerce')
original_data['adjusted_percentage_of_average_daily_traffic_for_trucks']=original_data['adjusted_percentage_of_average_daily_traffic_for_trucks'].apply(pd.to_numeric, errors='coerce')
original_data['longitude']=original_data['longitude'].apply(pd.to_numeric, errors='coerce')
original_data['latitude']=original_data['latitude'].apply(pd.to_numeric, errors='coerce')
original_data['median_width']=original_data['median_width'].apply(pd.to_numeric, errors='coerce')
original_data['number_of_lanes']=original_data['number_of_lanes'].apply(pd.to_numeric, errors='coerce')
original_data['right_shoulder_width']=original_data['right_shoulder_width'].apply(pd.to_numeric, errors='coerce')
#check results
print(original_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8859 entries, 0 to 13947
Data columns (total 58 columns):
crash_id                                                   8859 non-null int64
active_school_zone_flag                                    8859 non-null object
adjusted_average_daily_traffic_amount                      8859 non-null int64
adjusted_percentage_of_average_daily_traffic_for_trucks    8859 non-null float64
at_intersection_flag                                       8859 non-null bool
commercial_motor_vehicle_flag                              8859 non-null object
construction_zone_flag                                     8859 non-null object
county                                                     8859 non-null object
crash_severity                                             8859 non-null object
crash_time                                                 8859 non-null int64
crash_year                                                 8859 non-null int64
curve_degrees    

In [5]:
#all condidered features, removed some due to large portion of NA
crash_features = [ 'active_school_zone_flag',
       'adjusted_average_daily_traffic_amount',
       'adjusted_percentage_of_average_daily_traffic_for_trucks',
       'at_intersection_flag', 'commercial_motor_vehicle_flag',
       'construction_zone_flag', 'county', 'crash_severity', 'crash_time',
       'crash_year', 
       'day_of_week', 'first_harmful_event', 'highway_lane_design',
       'inside_shoulder_width_on_divided_highway', 'intersection_related',
       'latitude', 'left_shoulder_type', 'left_shoulder_use',
       'left_shoulder_width', 'light_condition', 'longitude',
       'manner_of_collision', 'median_type', 'median_width', 'number_of_lanes',
       'right_shoulder_type', 'right_shoulder_use',
       'right_shoulder_width', 'road_class', 'roadway_alignment',
       'roadway_function', 'roadway_part', 'roadway_type', 
       'speed_limit', 'surface_type', 
       'traffic_control_type', 'weather_condition', 'contributing_factor_1',
       'driver_license_class', 'driver_license_type', 'vehicle_body_style',
       'vehicle_travel_direction', 'person_age', 'person_ethnicity',
       'person_gender', 'person_restraint_used']

X = original_data[crash_features]
X.head() #display top few rows 

,active_school_zone_flag,adjusted_average_daily_traffic_amount,adjusted_percentage_of_average_daily_traffic_for_trucks,at_intersection_flag,commercial_motor_vehicle_flag,construction_zone_flag,county,crash_severity,crash_time,crash_year,day_of_week,first_harmful_event,highway_lane_design,inside_shoulder_width_on_divided_highway,intersection_related,latitude,left_shoulder_type,left_shoulder_use,left_shoulder_width,light_condition,longitude,manner_of_collision,median_type,median_width,number_of_lanes,right_shoulder_type,right_shoulder_use,right_shoulder_width,road_class,roadway_alignment,roadway_function,roadway_part,roadway_type,speed_limit,surface_type,traffic_control_type,weather_condition,contributing_factor_1,driver_license_class,driver_license_type,vehicle_body_style,vehicle_travel_direction,person_age,person_ethnicity,person_gender,person_restraint_used
0,No,23336,35.0,False,No,No,Callahan,N - NOT INJURED,929,2010,Saturday,Motor Vehicle In Transport,Freeway,4,Driveway Access,32.412067,Surfaced,Emergency Only,8,Daylight,-99.490320,Angle - One Straight-One Left Turn,Positive Barrier,40,4,Surfaced,Emergency Only,20,Interstate,"Straight, Level",Rural Interstate,Service/Frontage Road,"4 Or More Lanes, Divided",45,No Data,No Passing Zone,Rain,None,Class C,Driver License,"Passenger Car, 2-Door",East,16,White,Female,Shoulder & Lap Belt
1,No,28778,31.4,False,Yes,No,Callahan,N - NOT INJURED,2025,2010,Friday,Motor Vehicle In Transport,Freeway,4,Non Intersection,32.420931,Surfaced,Emergency Only,8,"Dark, Not Lighted",-99.540594,Same Direction - Both Going Straight-Sideswipe,Positive Barrier,40,4,Surfaced,Emergency Only,20,Interstate,"Straight, Grade",Rural Interstate,Main/Proper Lane,"4 Or More Lanes, Divided",65,No Data,Marked Lanes,Cloudy,Failed To Control Speed,Class C,Driver License,"Passenger Car, 2-Door",West,19,White,Male,Shoulder & Lap Belt
2,No,19832,38.2,False,No,No,Callahan,B - NON-INCAPACITATING INJURY,1318,2010,Sunday,Fixed Object,Freeway,4,Non Intersection,32.384917,Surfaced,Emergency Only,8,Daylight,-99.290845,One Motor Vehicle - Going Straight,Positive Barrier,40,4,Surfaced,Emergency Only,20,Interstate,"Straight, Level",Rural Interstate,Main/Proper Lane,"4 Or More Lanes, Divided",70,No Data,Center Stripe/Divider,Fog,Failed To Drive In Single Lane,Class C,Driver License,"Passenger Car, 4-Door",East,18,Black,Female,Shoulder & Lap Belt
3,No,28778,31.4,False,No,No,Callahan,N - NOT INJURED,850,2010,Thursday,Fixed Object,Freeway,4,Non Intersection,32.419109,Surfaced,Emergency Only,8,Daylight,-99.530243,One Motor Vehicle - Going Straight,Positive Barrier,40,4,Surfaced,Emergency Only,20,Interstate,"Straight, Level",Rural Interstate,Main/Proper Lane,"4 Or More Lanes, Divided",70,No Data,None,Snow,Other (Explain In Narrative),Class C,Driver License,Sport Utility Vehicle,West,20,White,Male,Shoulder & Lap Belt
4,No,23336,35.0,False,No,No,Callahan,N - NOT INJURED,1854,2010,Sunday,Fixed Object,Freeway,4,Non Intersection,32.409016,Surfaced,Emergency Only,8,"Dark, Not Lighted",-99.466104,One Motor Vehicle - Going Straight,Positive Barrier,40,4,Surfaced,Emergency Only,20,Interstate,"Straight, Level",Rural Interstate,Main/Proper Lane,"4 Or More Lanes, Divided",65,No Data,Center Stripe/Divider,Clear,None,Class C,Driver License,"Passenger Car, 2-Door",West,18,Hispanic,Female,Shoulder & Lap Belt


In [6]:
features = X.copy() #make a copy
print(features.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8859 entries, 0 to 13947
Data columns (total 46 columns):
active_school_zone_flag                                    8859 non-null object
adjusted_average_daily_traffic_amount                      8859 non-null int64
adjusted_percentage_of_average_daily_traffic_for_trucks    8859 non-null float64
at_intersection_flag                                       8859 non-null bool
commercial_motor_vehicle_flag                              8859 non-null object
construction_zone_flag                                     8859 non-null object
county                                                     8859 non-null object
crash_severity                                             8859 non-null object
crash_time                                                 8859 non-null int64
crash_year                                                 8859 non-null int64
day_of_week                                                8859 non-null object
first_harmful_ev

In [7]:
#function for timer, return the running time
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

label encode categorical features

In [8]:
#label encode crash_severity (target) manually
cleanup = {"crash_severity": {"N - NOT INJURED": 0, "B - NON-INCAPACITATING INJURY": 1, "C - POSSIBLE INJURY": 2, 
                             "A - SUSPECTED SERIOUS INJURY": 3, "K - KILLED": 4}}
features.replace(cleanup, inplace=True)
target = features["crash_severity"] 
features.drop(["crash_severity"], inplace=True, axis=1)
feature_names = features.columns.tolist()
#divide columns into numerical/categorical
categorical_subset = features.select_dtypes("object")
#label encode other categorical features
label_fields = list(categorical_subset.columns)
label_encoder = preprocessing.LabelEncoder() 
features[label_fields] = label_encoder.fit_transform(label_fields)
features.head()

,active_school_zone_flag,adjusted_average_daily_traffic_amount,adjusted_percentage_of_average_daily_traffic_for_trucks,at_intersection_flag,commercial_motor_vehicle_flag,construction_zone_flag,county,crash_time,crash_year,day_of_week,first_harmful_event,highway_lane_design,inside_shoulder_width_on_divided_highway,intersection_related,latitude,left_shoulder_type,left_shoulder_use,left_shoulder_width,light_condition,longitude,manner_of_collision,median_type,median_width,number_of_lanes,right_shoulder_type,right_shoulder_use,right_shoulder_width,road_class,roadway_alignment,roadway_function,roadway_part,roadway_type,speed_limit,surface_type,traffic_control_type,weather_condition,contributing_factor_1,driver_license_class,driver_license_type,vehicle_body_style,vehicle_travel_direction,person_age,person_ethnicity,person_gender,person_restraint_used
0,0,23336,35.0,False,1,2,4,929,2010,5,8,9,10,11,32.412067,12,13,14,15,-99.490320,16,17,40,4,21,22,20,23,24,25,26,27,45,28,29,32,3,6,7,30,31,16,18,19,20
1,0,28778,31.4,False,1,2,4,2025,2010,5,8,9,10,11,32.420931,12,13,14,15,-99.540594,16,17,40,4,21,22,20,23,24,25,26,27,65,28,29,32,3,6,7,30,31,19,18,19,20
2,0,19832,38.2,False,1,2,4,1318,2010,5,8,9,10,11,32.384917,12,13,14,15,-99.290845,16,17,40,4,21,22,20,23,24,25,26,27,70,28,29,32,3,6,7,30,31,18,18,19,20
3,0,28778,31.4,False,1,2,4,850,2010,5,8,9,10,11,32.419109,12,13,14,15,-99.530243,16,17,40,4,21,22,20,23,24,25,26,27,70,28,29,32,3,6,7,30,31,20,18,19,20
4,0,23336,35.0,False,1,2,4,1854,2010,5,8,9,10,11,32.409016,12,13,14,15,-99.466104,16,17,40,4,21,22,20,23,24,25,26,27,65,28,29,32,3,6,7,30,31,18,18,19,20


In [9]:
float_features = features.xs(feature_names,axis=1).values
#standardize the continuous features
scaler = StandardScaler()
float_scaled = scaler.fit_transform(float_features)
features[feature_names] = float_scaled
features.head()

C:\Users\amand\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\amand\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,active_school_zone_flag,adjusted_average_daily_traffic_amount,adjusted_percentage_of_average_daily_traffic_for_trucks,at_intersection_flag,commercial_motor_vehicle_flag,construction_zone_flag,county,crash_time,crash_year,day_of_week,first_harmful_event,highway_lane_design,inside_shoulder_width_on_divided_highway,intersection_related,latitude,left_shoulder_type,left_shoulder_use,left_shoulder_width,light_condition,longitude,manner_of_collision,median_type,median_width,number_of_lanes,right_shoulder_type,right_shoulder_use,right_shoulder_width,road_class,roadway_alignment,roadway_function,roadway_part,roadway_type,speed_limit,surface_type,traffic_control_type,weather_condition,contributing_factor_1,driver_license_class,driver_license_type,vehicle_body_style,vehicle_travel_direction,person_age,person_ethnicity,person_gender,person_restraint_used
0,0.0,0.700426,1.648150,-0.624392,0.0,0.0,0.0,-0.765932,-1.665485,0.0,0.0,0.0,0.0,0.0,-0.375391,0.0,0.0,0.0,0.0,0.341953,0.0,0.0,0.692618,0.837729,0.0,0.0,1.217619,0.0,0.0,0.0,0.0,0.0,-1.116884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.686390,0.0,0.0,0.0
1,0.0,1.066361,1.338284,-0.624392,0.0,0.0,0.0,1.151681,-1.665485,0.0,0.0,0.0,0.0,0.0,-0.366863,0.0,0.0,0.0,0.0,0.315209,0.0,0.0,0.692618,0.837729,0.0,0.0,1.217619,0.0,0.0,0.0,0.0,0.0,0.273317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.526138,0.0,0.0,0.0
2,0.0,0.464807,1.923587,-0.624392,0.0,0.0,0.0,-0.085319,-1.665485,0.0,0.0,0.0,0.0,0.0,-0.401512,0.0,0.0,0.0,0.0,0.448067,0.0,0.0,0.692618,0.837729,0.0,0.0,1.217619,0.0,0.0,0.0,0.0,0.0,0.620867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.211371,0.0,0.0,0.0
3,0.0,1.066361,1.338284,-0.624392,0.0,0.0,0.0,-0.904154,-1.665485,0.0,0.0,0.0,0.0,0.0,-0.368616,0.0,0.0,0.0,0.0,0.320716,0.0,0.0,0.692618,0.837729,0.0,0.0,1.217619,0.0,0.0,0.0,0.0,0.0,0.620867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.263648,0.0,0.0,0.0
4,0.0,0.700426,1.648150,-0.624392,0.0,0.0,0.0,0.852492,-1.665485,0.0,0.0,0.0,0.0,0.0,-0.378327,0.0,0.0,0.0,0.0,0.354835,0.0,0.0,0.692618,0.837729,0.0,0.0,1.217619,0.0,0.0,0.0,0.0,0.0,0.273317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.211371,0.0,0.0,0.0


In [10]:
y = target.values
X = features.values
#split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)
feature_sel = range(len(feature_names))
fnames = np.array(feature_names)[feature_sel]

train model

In [11]:
#convert train and test sets to DMatrix (input of xgboost data set)
dtrain = xgb.DMatrix(X_train, label=y_train,feature_names=fnames)
dtest = xgb.DMatrix(X_test, label=y_test,feature_names=fnames)
#baseline prediction in mean absolute error
mean_train = np.mean(y_train)
baseline_predictions = np.ones(y_test.shape) * mean_train
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE is {:.10f}".format(mae_baseline))

Baseline MAE is 0.8053601206


In [12]:
params = {
    #parameters that we are going to tune.
    'max_depth':5,
    'min_child_weight': 3,
    'eta':.2,
    'subsample': 1,
    'colsample_bytree': 1,
    #other parameter (linear regression)
    'objective':'reg:linear',
}
#evaluation metric: mean absolute error
params['eval_metric'] = "mae"
num_boost_round = 999
start_time = timer(None)
#train the model
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
timer(start_time)
print("Best MAE: {:.10f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

[0]	Test-mae:0.778846
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:0.779267
[2]	Test-mae:0.779985
[3]	Test-mae:0.78032
[4]	Test-mae:0.780262
[5]	Test-mae:0.780876
[6]	Test-mae:0.780577
[7]	Test-mae:0.780781
[8]	Test-mae:0.780482
[9]	Test-mae:0.780845
[10]	Test-mae:0.780726
Stopping. Best iteration:
[0]	Test-mae:0.778846


 Time taken: 0 hours 0 minutes and 1.47 seconds.
Best MAE: 0.7788460000 with 1 rounds


xgboost hyperparameter tuning

In [13]:
#use auc as evaluation metric or further hyperparameter tuning
params['eval_metric'] = "auc"
num_boost_round = 999
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best AUC: {:.10f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

[0]	Test-auc:0.611863
Will train until Test-auc hasn't improved in 10 rounds.
[1]	Test-auc:0.639718
[2]	Test-auc:0.659855
[3]	Test-auc:0.670064
[4]	Test-auc:0.673281
[5]	Test-auc:0.674024
[6]	Test-auc:0.674346
[7]	Test-auc:0.674486
[8]	Test-auc:0.676654
[9]	Test-auc:0.677333
[10]	Test-auc:0.680022
[11]	Test-auc:0.67875
[12]	Test-auc:0.678786
[13]	Test-auc:0.678739
[14]	Test-auc:0.677338
[15]	Test-auc:0.674746
[16]	Test-auc:0.673886
[17]	Test-auc:0.674062
[18]	Test-auc:0.679412
[19]	Test-auc:0.681129
[20]	Test-auc:0.682521
[21]	Test-auc:0.680416
[22]	Test-auc:0.679942
[23]	Test-auc:0.682119
[24]	Test-auc:0.69035
[25]	Test-auc:0.68908
[26]	Test-auc:0.683607
[27]	Test-auc:0.680236
[28]	Test-auc:0.679342
[29]	Test-auc:0.679594
[30]	Test-auc:0.678124
[31]	Test-auc:0.674995
[32]	Test-auc:0.674069
[33]	Test-auc:0.672413
[34]	Test-auc:0.66705
Stopping. Best iteration:
[24]	Test-auc:0.69035

Best AUC: 0.6903500000 with 25 rounds


In [14]:
#tune the pair of max_depth and min_child_weight
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

In [15]:
max_auc = 0
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    #update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    #run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )
    #update best AUC
    mean_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, AUC: {}".format(best_params[0], best_params[1], max_auc))

CV with max_depth=9, min_child_weight=5


C:\Users\amand\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


	AUC 0.6571431999999999 for 8 rounds
CV with max_depth=9, min_child_weight=6
	AUC 0.6586858 for 9 rounds
CV with max_depth=9, min_child_weight=7
	AUC 0.6637519999999999 for 12 rounds
CV with max_depth=10, min_child_weight=5
	AUC 0.6512154000000001 for 26 rounds
CV with max_depth=10, min_child_weight=6
	AUC 0.6548102 for 7 rounds
CV with max_depth=10, min_child_weight=7
	AUC 0.6646378000000001 for 12 rounds
CV with max_depth=11, min_child_weight=5
	AUC 0.6537054 for 15 rounds
CV with max_depth=11, min_child_weight=6
	AUC 0.6533346000000001 for 20 rounds
CV with max_depth=11, min_child_weight=7
	AUC 0.6724817999999999 for 21 rounds
Best params: 11, 7, AUC: 0.6724817999999999


In [16]:
#update the best values for the pair above
params['max_depth'] = 9
params['min_child_weight'] = 6

In [17]:
#tune the pair of subsample and colsample
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]
max_auc = 0
best_params = None
#start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    #update parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    #run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )
    #update best score
    max_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = (subsample,colsample)
print("Best params: {}, {}, AUC: {}".format(best_params[0], best_params[1], max_auc))

CV with subsample=1.0, colsample=1.0


C:\Users\amand\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


	AUC 0.6724817999999999 for 9 rounds
CV with subsample=1.0, colsample=0.9
	AUC 0.6724817999999999 for 19 rounds
CV with subsample=1.0, colsample=0.8
	AUC 0.6724817999999999 for 14 rounds
CV with subsample=1.0, colsample=0.7
	AUC 0.6724817999999999 for 4 rounds
CV with subsample=0.9, colsample=1.0
	AUC 0.6724817999999999 for 8 rounds
CV with subsample=0.9, colsample=0.9
	AUC 0.6724817999999999 for 9 rounds
CV with subsample=0.9, colsample=0.8
	AUC 0.6724817999999999 for 10 rounds
CV with subsample=0.9, colsample=0.7
	AUC 0.6724817999999999 for 33 rounds
CV with subsample=0.8, colsample=1.0
	AUC 0.6724817999999999 for 9 rounds
CV with subsample=0.8, colsample=0.9
	AUC 0.6724817999999999 for 7 rounds
CV with subsample=0.8, colsample=0.8
	AUC 0.6724817999999999 for 30 rounds
CV with subsample=0.8, colsample=0.7
	AUC 0.6724817999999999 for 19 rounds
CV with subsample=0.7, colsample=1.0
	AUC 0.6724817999999999 for 5 rounds
CV with subsample=0.7, colsample=0.9
	AUC 0.6724817999999999 for 5 ro

In [18]:
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7

In [19]:
#tune eta
max_auc = 0
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    #update parameters
    params['eta'] = eta
    #run and time each CV
    %time cv_results = xgb.cv(params, dtrain, num_boost_round=num_boost_round, seed=42, nfold=5, metrics=['auc'], early_stopping_rounds=10)
    #update best score
    mean_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds\n".format(mean_auc, boost_rounds))
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = eta
print("Best params: {}, AUC: {}".format(best_params, max_auc))

CV with eta=0.3
Wall time: 1.96 s
	AUC 0.6261314 for 17 rounds

CV with eta=0.2


C:\Users\amand\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  if sys.path[0] == '':


Wall time: 1.68 s
	AUC 0.6607176000000001 for 9 rounds

CV with eta=0.1
Wall time: 3.27 s
	AUC 0.6672554 for 31 rounds

CV with eta=0.05
Wall time: 2.15 s
	AUC 0.6687108 for 20 rounds

CV with eta=0.01
Wall time: 1.7 s
	AUC 0.668645 for 11 rounds

CV with eta=0.005
Wall time: 1.72 s
	AUC 0.6759636 for 11 rounds

Best params: 0.005, AUC: 0.6759636


In [20]:
params['eta'] = .005
#check all parameters tuned
params

{'max_depth': 9,
 'min_child_weight': 6,
 'eta': 0.005,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'objective': 'reg:linear',
 'eval_metric': 'auc'}

In [21]:
#train the model with tuned parameters
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Best AUC: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

[0]	Test-auc:0.589349
Will train until Test-auc hasn't improved in 10 rounds.
[1]	Test-auc:0.630162
[2]	Test-auc:0.654251
[3]	Test-auc:0.651093
[4]	Test-auc:0.654633
[5]	Test-auc:0.665478
[6]	Test-auc:0.655843
[7]	Test-auc:0.659019
[8]	Test-auc:0.660855
[9]	Test-auc:0.662988
[10]	Test-auc:0.667204
[11]	Test-auc:0.661403
[12]	Test-auc:0.665108
[13]	Test-auc:0.665631
[14]	Test-auc:0.666815
[15]	Test-auc:0.670262
[16]	Test-auc:0.673446
[17]	Test-auc:0.672953
[18]	Test-auc:0.675317
[19]	Test-auc:0.675401
[20]	Test-auc:0.674411
[21]	Test-auc:0.676922
[22]	Test-auc:0.680581
[23]	Test-auc:0.681604
[24]	Test-auc:0.680848
[25]	Test-auc:0.684867
[26]	Test-auc:0.685164
[27]	Test-auc:0.687123
[28]	Test-auc:0.689838
[29]	Test-auc:0.691097
[30]	Test-auc:0.689992
[31]	Test-auc:0.692399
[32]	Test-auc:0.695763
[33]	Test-auc:0.694381
[34]	Test-auc:0.695349
[35]	Test-auc:0.69458
[36]	Test-auc:0.694182
[37]	Test-auc:0.696659
[38]	Test-auc:0.695974
[39]	Test-auc:0.695063
[40]	Test-auc:0.695501
[41]	Test-au

In [22]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-auc:0.589349
[1]	Test-auc:0.630162
[2]	Test-auc:0.654251
[3]	Test-auc:0.651093
[4]	Test-auc:0.654633
[5]	Test-auc:0.665478
[6]	Test-auc:0.655843
[7]	Test-auc:0.659019
[8]	Test-auc:0.660855
[9]	Test-auc:0.662988
[10]	Test-auc:0.667204
[11]	Test-auc:0.661403
[12]	Test-auc:0.665108
[13]	Test-auc:0.665631
[14]	Test-auc:0.666815
[15]	Test-auc:0.670262
[16]	Test-auc:0.673446
[17]	Test-auc:0.672953
[18]	Test-auc:0.675317
[19]	Test-auc:0.675401
[20]	Test-auc:0.674411
[21]	Test-auc:0.676922
[22]	Test-auc:0.680581
[23]	Test-auc:0.681604
[24]	Test-auc:0.680848
[25]	Test-auc:0.684867
[26]	Test-auc:0.685164
[27]	Test-auc:0.687123
[28]	Test-auc:0.689838
[29]	Test-auc:0.691097
[30]	Test-auc:0.689992
[31]	Test-auc:0.692399
[32]	Test-auc:0.695763
[33]	Test-auc:0.694381
[34]	Test-auc:0.695349
[35]	Test-auc:0.69458
[36]	Test-auc:0.694182
[37]	Test-auc:0.696659


Even after hyperparameter tuning, AUC score is still not good. 

In [23]:
#plt.figure(figsize=(15,15))
#xgb.plot_importance(best_model,ax=plt.gca(),importance_type='weight')

In [24]:
#best_model.save_model("label_encoding_xgboost.model")